In [1]:
# Dependencies
import pandas as pd 
import requests
import json
import random

# Google API Key
from config import gkey

file_path = "Resources/winemag-data_first150k.csv"
data = pd.read_csv("Resources/winemag-data_first150k.csv") 

# Preview the first 50 lines of the loaded data
data.head(50)


Unnamed: 0      country  \
0            0           US   
1            1        Spain   
2            2           US   
3            3           US   
4            4       France   
5            5        Spain   
6            6        Spain   
7            7        Spain   
8            8           US   
9            9           US   
10          10        Italy   
11          11           US   
12          12           US   
13          13       France   
14          14           US   
15          15           US   
16          16           US   
17          17        Spain   
18          18       France   
19          19           US   
20          20           US   
21          21        Spain   
22          22        Spain   
23          23           US   
24          24           US   
25          25  New Zealand   
26          26           US   
27          27           US   
28          28           US   
29          29           US   
30          30     Bulgaria   
31          31           US   
32          32        Italy   
33          33       France   
34          34           US   
35          35        Italy   
36          36       France   
37          37        Italy   
38          38        Italy   
39          39        Italy   
40          40        Spain   
41          41        Spain   
42          42           US   
43          43        Italy   
44          44       France   
45          45        Italy   
46          46        Italy   
47          47           US   
48          48        Italy   
49          49           US   

                                          description  \
0   This tremendous 100% varietal wine hails from ...   
1   Ripe aromas of fig, blackberry and cassis are ...   
2   Mac Watson honors the memory of a wine once ma...   
3   This spent 20 months in 30% new French oak, an...   
4   This is the top wine from La Bégude, named aft...   
5   Deep, dense and pure from the opening bell, th...   
6   Slightly gritty black-fruit aromas include a s...   
7   Lush cedary black-fruit aromas are luxe and of...   
8   This re-named vineyard was formerly bottled as...   
9   The producer sources from two blocks of the vi...   
10  Elegance, complexity and structure come togeth...   
11  From 18-year-old vines, this supple well-balan...   
12  A standout even in this terrific lineup of 201...   
13  This wine is in peak condition. The tannins an...   
14  With its sophisticated mix of mineral, acid an...   
15  First made in 2006, this succulent luscious Ch...   
16  This blockbuster, powerhouse of a wine suggest...   
17  Nicely oaked blackberry, licorice, vanilla and...   
18  Coming from a seven-acre vineyard named after ...   
19  This fresh and lively medium-bodied wine is be...   
20  Heitz has made this stellar rosé from the rare...   
21  Alluring, complex and powerful aromas of grill...   
22  Tarry blackberry and cheesy oak aromas are app...   
23  The apogee of this ambitious winery's white wi...   
24  San Jose-based producer Adam Comartin heads 1,...   
25  Yields were down in 2015, but intensity is up,...   
26  Bergström has made a Shea designate since 2003...   
27  Focused and dense, this intense wine captures ...   
28  Cranberry, baked rhubarb, anise and crushed sl...   
29  This standout Rocks District wine brings earth...   
30  This Bulgarian Mavrud presents the nose with s...   
31  Steely and perfumed, this wine sees only 20% n...   
32  Underbrush, scorched earth, menthol and plum s...   
33  Pale in color, this is nutty in character, wit...   
34  The aromas entice with notes of wet stone, hon...   
35  Forest floor, tilled soil, mature berry and a ...   
36  Gingery spice notes accent fresh pear and melo...   
37  Aromas of forest floor, violet, red berry and ...   
38  This has a charming nose that boasts rose, vio...   
39  This bright, savory wine delivers aromas and f...   
40  Earthy plum and cherry aromas score points for...   
41  This comes across wide a

In [2]:
# Drop columns no needed
data.drop(columns=["Unnamed: 0", "description", "region_2"], inplace=True)

# Remove missing values
data.dropna(inplace=True)

In [4]:
# Sample for project purpose
sample = data.sample(n=10, axis=0, random_state = 4)
sample.reset_index(drop = True, inplace=True)

sample

country              designation  points  price           province  \
0  Australia                    Thorn      88   19.0    South Australia   
1      Italy  Doganella Il Tintorosso      87   15.0            Tuscany   
2         US                  Reserve      87   28.0         California   
3         US       Ellenbach Vineyard      90   60.0         California   
4      Italy           Il Monovitigno      88   30.0  Sicily & Sardinia   
5      Italy                  Centine      87   12.0            Tuscany   
6      Spain                  Crianza      85   12.0      Central Spain   
7         US                    Gypsy      90   25.0         California   
8         US                  Reserve      89   24.0         California   
9      Italy                 Le Vigne      86  118.0           Piedmont   

               region_1                variety           winery  
0           Eden Valley               Riesling      Grant Burge  
1               Maremma              Red Blend          Tommasi  
2           Paso Robles     Cabernet Sauvignon  Castoro Cellars  
3          Sonoma Coast             Pinot Noir            Etude  
4               Sicilia           Nero d'Avola          Benanti  
5               Toscana                   Rosé            Banfi  
6             La Mancha              Red Blend    Finca Antigua  
7  Santa Barbara County  Rhône-style Red Blend         Epiphany  
8      Livermore Valley                  Syrah        Concannon  
9                Barolo               Nebbiolo         Sandrone

In [7]:
sample["latitud"] = ""
sample["longitud"] = ""
sample["elevation"] = ""


# Build URL using the Google Geocoding API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# Iterate dataframe for filling Latitud and Longitud columns
for index, row in sample.iterrows():
    
    # Build winery location search
    winery_loc = row["winery"] + "," + row["region_1"]  + "," +  row["province"]
    
    # Parameters for API request
    params = {"address": winery_loc, "key": gkey}
    
    try:

        # Run request
        response = requests.get(base_url, params=params)

        # Convert to JSON
        geo_data = response.json()

        # Extract lat/lng and fill columns
        sample.loc[index, 'latitud'] = geo_data["results"][0]["geometry"]["location"]["lat"]
        sample.loc[index, 'longitud'] = geo_data["results"][0]["geometry"]["location"]["lng"]
    
        # print Winery location found
        print(f"Winery location {winery_loc} {sample.loc[index, 'latitud']} {sample.loc[index, 'longitud']} found")
        
        
    except (KeyError, IndexError):
        print("Winery not found" )
        
#test.head(10)

# Iterate dataframe for filling Elevation column
for index, row in sample.iterrows():
    lat = row["latitud"]
    lng = row["longitud"]
    
    try:
        # Build URL using the Google Elevation API
        url_ele = f"https://maps.googleapis.com/maps/api/elevation/json?locations={lat},{lng}&key={gkey}"
        
        # Run request
        response = requests.get(url_ele)

        # Convert to JSON
        geo_data = response.json()

        # Extract elevation and fill column
        sample.loc[index, 'elevation'] = geo_data["results"][0]["elevation"]
    
        # print Winery elevation found
        print(f"Winery elevation {sample.loc[index, 'elevation']} found")
        
        
    except (KeyError, IndexError):
        print("Elevation not found" )

#test.head(10)

sample.to_csv("Resources/winery_data.csv", index=False)


Winery location Grant Burge,Eden Valley,South Australia -34.5615916 138.9754373 found
Winery location Tommasi,Maremma,Tuscany 42.6086111 11.6966667 found
Winery location Castoro Cellars,Paso Robles,California 35.5772607 -120.722921 found
Winery location Etude,Sonoma Coast,California 38.2458215 -122.3150228 found
Winery location Benanti,Sicilia,Sicily & Sardinia 37.5999938 14.0153557 found
Winery location Banfi,Toscana,Tuscany 42.980621 11.400659 found
Winery location Finca Antigua,La Mancha,Central Spain 39.6310835 -2.8920419 found
Winery location Epiphany,Santa Barbara County,California 34.6669689 -120.1147479 found
Winery location Concannon,Livermore Valley,California 37.666199 -121.7397388 found
Winery location Sandrone,Barolo,Piedmont 44.6208903 7.954625300000001 found
Winery elevation 323.2424926757812 found
Winery elevation 354.9888916015625 found
Winery elevation 280.32470703125 found
Winery elevation 16.87372779846191 found
Winery elevation 622.5406494140625 found
Winery elevat